# Some TensorFlow 2.0 and TensorFlow Probability basics

Based on: https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/A_Tour_of_TensorFlow_Probability.ipynb

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

## TensorFlow: linear algebra

As long as there are no for loops, all the operations are automatically vectorizes (and thus more performant).

Solving a linear system
$$
y = m x
$$
for $x$.

In [ ]:
m = tf.random.uniform(shape=[10, 10])

y = tf.random.uniform(shape=[10, 1])

In [ ]:
x = tf.linalg.solve(
    matrix=m,
    rhs=y
)

In [ ]:
np.allclose(
    tf.linalg.matmul(m, x).numpy(),
    y.numpy()
)

This can of course be achieved by inverting $m$ by hand (if it's invertible!).

In [ ]:
if tf.linalg.det(m) != 0:
    minv = tf.linalg.inv(m)
    
    x_alternative = tf.linalg.matmul(minv, y)

else:
    print("Matrix m is not invertible!")

In [ ]:
np.allclose(
    x_alternative.numpy(),
    x.numpy()
)

We can also define sets of tensors stacking them teogether using another dimension.

In [ ]:
m_stacked = tf.random.uniform(shape=(5, 10, 10))

y_stacked = tf.random.uniform(shape=(5, 10, 1))

In [ ]:
# Invert each matrix.
m_stacked_inv = tf.linalg.inv(m_stacked)

m_stacked_inv.shape

In [ ]:
x_stacked = tf.linalg.matmul(m_stacked_inv, y_stacked)

x_stacked.shape

## Automatic differentiation

Differentiating
$$
b = \frac{1}{2}\,a^2
$$
w.r.t. $a$ and setting $a = 29$.

In [ ]:
a = tf.constant(29.)

with tf.GradientTape() as tape:
    tape.watch([a])
    
    b = 0.5 * a**2
    
grad = tape.gradient(b, a)

grad

Differentiating w.r.t. multiple variables (proper gradient) the function
$$
F(a, b) = a\,\sin^b(b)
$$
and setting $a=1$ and $b=\pi/2$.

Result:
$$
\nabla F(a, b) = \left(\begin{array}{c}
\sin^b(b) \\
a\,b \cos(b)
\end{array}\right) =
\left(\begin{array}{c}
1 \\
0
\end{array}\right)
$$

In [ ]:
a = tf.constant(1.)
b = tf.constant(np.pi/2.)

with tf.GradientTape() as tape:
    tape.watch([a, b])
    
    f = a * (tf.sin(b)) ** b

grad = tape.gradient(f, [a, b])

grad

Differentiating w.r.t. vectors: let

$$
\mathbf{a} = \left(\begin{array}{r}
1 \\
2 \\
3
\end{array}\right) \equiv
\left(\begin{array}{r}
a_1 \\
a_2 \\
a_3
\end{array}\right)\in \mathbb{R}^3,\quad
b = \left(\begin{array}{ccc}
1 & 0 & 0\\
0 & -1 & 0 \\
0 & 0 & 1
\end{array}\right) \equiv
\left(\begin{array}{ccc}
b_{11} & b_{12} & b_{13} \\
b_{21} & b_{22} & b_{23} \\
b_{31} & b_{32} & b_{33}
\end{array}\right)\in \text{Mat}_{\mathbb{R}}(3)
$$

and consider the differentiation of the product $h(\mathbf{a}) = b\,\mathbf{a}$ w.r.t. (the components of) $\mathbf{a}$, setting $\mathbf{a}$ to the above value in the end

$$
\nabla h(\mathbf{a}) = \left(\begin{array}{r}
\partial_{a_1} h(a_1, a_2, a_3) \\
\partial_{a_2} h(a_1, a_2, a_3) \\
\partial_{a_3} h(a_1, a_2, a_3)
\end{array}\right) =
\left(\begin{array}{r}
b_{11} \\
b_{22} \\
b_{33}
\end{array}\right) =
\left(\begin{array}{r}
1 \\
-1 \\
1
\end{array}\right)
$$

In [ ]:
a = tf.constant([[1], [2], [3]], dtype=tf.float32)

b = tf.constant([
    [1, 0, 0],
    [0, -1, 0],
    [0, 0, 1]
], dtype=tf.float32)

with tf.GradientTape() as tape:
    tape.watch(a)
    
    g = tf.linalg.matmul(b, a)
    
grad = tape.gradient(g, a)

grad

Implement a gradient descent algorithm to find a minimum of

$$
F(x) = - x^2 + 2\,x^4,
$$

where the (global) minima are $x_{1, 2} = \pm1/2$. The algorithm will find either point, starting from $x=3$, where the value of the function is $1/8 = 0.125$.

In [ ]:
x = tf.constant(3, dtype=tf.float32)

eps = 0.001

n_iter = 1000

x_values = [x.numpy()]
f_values = []
der_values = []

for i in range(n_iter):
    # print(f"Iteration {i+1}")
    # print("-----------")
    
    with tf.GradientTape() as tape:
        tape.watch(x)

        f = - x ** 2 + 2. * x ** 4
        
        f_values.append(f.numpy())

    grad = tape.gradient(f, x)
    
    der_values.append(grad.numpy())

    # print(f"f'(x={x}): {grad}")

    x = x - grad * eps
    
    x_values.append(x.numpy())

    # print(f"x_new: {x}\n")

f_values.append((- x ** 2 + 2. * x ** 4).numpy())

print("Final values")
print("------------")
print(f"(x, f(x)) = ({x_values[-1]}, {f_values[-1]}), f'(x) = {der_values[-1]}")

In [ ]:
trace = go.Scatter(
    x=list(range(len(x_values))),
    y=x_values,
    mode="markers"
)

fig = go.Figure(data=[trace])

iplot(fig)

In [ ]:
trace = go.Scatter(
    x=x_values,
    y=f_values,
    mode="markers",
    marker=dict(
        opacity=np.linspace(0.5, 1, len(x_values)),
    ),
)

fig = go.Figure(data=[trace])

iplot(fig)

## TensorFlow probability

### Distributions

The fundamental low-level object in TensorFlow Probability is the distribution, effectively a random number generator object. The two main mehtods of this class are `sample()` and `log_prob()`.

In [ ]:
tfd = tfp.distributions

In [ ]:
normal = tfd.Normal(loc=0., scale=1.)

Sample the normal distribution object.

In [ ]:
normal_samples = normal.sample(10000)

In [ ]:
trace = go.Histogram(
    x=normal_samples.numpy(),
    histnorm="probability"
)

fig = go.Figure(data=[trace])

iplot(fig)

Compute the log probability of a point in the domain of the distribution.

In [ ]:
normal.log_prob(0.)

Because the distribution is centered on 0 and symmetric, points that are symmetric w.r.t. 0 will have the same log probability.

In [ ]:
normal.log_prob([-1., 0., 1.])

Distributions distinguish between a __batch shape__ and an __event shape__:
- Batch shape = shape of a collection of distributions (possibly with different parameters)
- Event shape = shape of a sample drawn from a distribution

Batch shapes are put "on the right" while event shapes are put "on the left".

If we pass multiple values for the parameters of a distribution, a batch of distributions is automatically created (possibly broadcasting the values passed for the other parameters - if any - if shapes don't match). E.g., if we pass three values $\mu_1, \mu_2$ and $\mu_3$ for the mean of a Normal distribution and only one value $\sigma$ for its standard deviation, we create the batch of three Normal distributions

$$
\left( \mathcal{N}\left(\mu_1, \sigma\right), \mathcal{N}\left(\mu_2, \sigma\right), \mathcal{N}\left(\mu_3, \sigma\right) \right)
$$

The distributions in the same batch are independent instances of the distribution class and the computations on them happen in parallel (as if they were vectorized arrays).

In [ ]:
normal_batch = tfd.Normal(loc=[-5., 0., 5.], scale=1.)

print(f"Batch shape: {normal_batch.batch_shape}")

If we then sample the batch of distribution $N$ times we get a tensor with shape $\left( N, 3 \right)$.

In [ ]:
normal_batch_samples = normal_batch.sample(10000)

print(f"Shape of the sample: {normal_batch_samples.shape}")

data = []

for i in range(normal_batch.batch_shape[0]):
    data.append(go.Histogram(
        x=normal_batch_samples[:, i],
        histnorm="probability"
    ))
    
iplot(go.Figure(data=data))

The shapes of the samples can be arbitrary, but the distribution they are drawn from is chosen according to the right-most index.

In [ ]:
normal_batch.sample((21, 17)).shape

If we compute the log probability of an array of value on a batch of distribution, with the array having the same shape of the batch, we get an array of log probabilities, each from a different distribution and for the corresponding value specified.

E.g. if we have three Normal distributions centered in -5, 0 and 5 respectively, and we compute the log probability of the array $(-5, 0, -5)$, we get the same value for all the entries.

In [ ]:
normal_batch.log_prob([-5., 0., 5.]).numpy()

On the other hand, if we compute the log probability of a scalar, that's broadcast into the batch shape and the log probability is returned for the same value on each of the distributions in the batch.

In [ ]:
normal_batch.log_prob(0.).numpy()